# Automatic Compilation for Neutral Atoms Quantum Circuits


### Imports

In [7]:
import utils
from utils import sep_print

import matplotlib
import sys

if 'ipykernel' in sys.modules:
    # Running in Jupyter
    pass  # Use default (inline) backend
else:
    matplotlib.use('TkAgg')  # or another suitable backend

import passes
import metrics
from validate import validate
from kirin.ir.method import Method

from bloqade import qasm2
from bloqade.qasm2.parse.lowering import QASM2
from bloqade.qasm2.passes import QASM2Py
from bloqade.qasm2.emit import QASM2 as QASM2Target # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

The following snippet imports all .qasm files from assets/baseline

In [ ]:
programs = utils.importQASM()

### Flags

These flags control the execution of the program, mainly which outputs are displyed and which optimizations passes are applied

In [ ]:
output_name = "1"           
# 1 is good
# 2 is bad also with NOTHING (even just with RydbergRewrite)
# 3 is bad with UToOpParallelise (commenting  nativeParallelise and with our passes brings to 1 fidelity)
# 4 is perfect
# 4_improved is perfect 

prettyDebug = False
printSSA = False

doRydberg = True
doNativeParallelisation = True

doOurPasses = False
doOurPasses_merge = True

Optionally output the initial qasm

In [ ]:
target = QASM2Target(allow_parallel=True)
program_ast = target.emit(programs[output_name])

if prettyDebug:
    sep_print("Non-translated qasm:\n")
    pprint(program_ast)


Select the circuit from the imports (in programs[]) and saves the initial state in qc_initial for the final fidelity evaluation

In [ ]:
circuit: Method = programs[output_name]
qc_initial = utils.circuit_to_qiskit(circuit)

## Passes
Rydberg translates from qasm to native gate set (U3 and CZ)

In [ ]:
if doRydberg:
    passes.RydbergRewrite(circuit)

print("Metrics after RydbergRewrite: ")
metrics.print_gate_counts(target.emit(circuit))

if printSSA:
    print("After Rydberg: ")
    circuit.print()

Remove2PiGates is our pass that removes 2pi rotations and meaningless U gates

In [ ]:
if doOurPasses:
    print("Doing Remove2PiGates Pass after RydbergRewrite...")
    passes.Remove2PiGates(circuit.dialects)(circuit)
if printSSA:
    circuit.print()

### Merge Pass
MergeConsecutiveU is our pass that merges U gates wherever possible to reduce their total number

In [ ]:
if doOurPasses_merge:
    print("Metrics before MERGE: ")
    metrics.print_gate_counts(target.emit(circuit))

    print("Merging ConsecutiveU")
    passes.MergeConsecutiveU(circuit.dialects)(circuit)

    print("Metrics after MERGE: ")
    metrics.print_gate_counts(target.emit(circuit))
if printSSA:
    print("circuit after MERGE: ")
    circuit.print()
    print()

In [ ]:
if prettyDebug:
    sep_print("Unparallelized QASMTarget:", sleepTimeSec=1)
    pprint(target.emit(circuit))

### Parallelisation Pass
The NativeParallelisationPass applies the native UOpToParallel pass to gather parallel CZ and U gates where possible

In [ ]:
if doNativeParallelisation:
    passes.NativeParallelisationPass(circuit)
    print("Metrics after nativeParallelise: ")
    metrics.print_gate_counts(target.emit(circuit))

if prettyDebug:
    sep_print("NativeParallelised circuit: ", sleepTimeSec=2)
    pprint(QASM2Target(allow_parallel=False).emit(circuit))


In [ ]:
if printSSA:
    circuit.print()

## Fidelity Evaluation
Finally, evaluates the fidelity of the final optimized circuit with respect to the original unoptimized version

In [ ]:
qc_final = utils.circuit_to_qiskit(circuit)

fidelity = validate(qc_initial, qc_final)

if fidelity > 0.8:
    filepath = f"../out_compiler/{output_name}.qasm" 
    print("Fidelity high enough. Exporting to QASM ", filepath)
    with open(filepath, "w") as out:
        out.write(target.emit_str(circuit))

Display the optimized circuit

In [ ]:
fig = qc_final.draw(output="mpl", fold=120, scale=0.7)
display(fig)